## Where People Seek Asylum

This is some basic cleaning for a small Tableau Visualization I did on [where people seek asylum](https://public.tableau.com/profile/eric.andresen#!/vizhome/WherePeopleSeekAsylum/Dashboard1).
The comes from the UNHCR's [monthly counts](http://popstats.unhcr.org/en/asylum_seekers_monthly) that are aggregations of national statistics.

In [29]:
import pandas as pd

In [37]:
seekers = pd.read_csv("unhcr_popstats_export_asylum_seekers_monthly_all_data.csv", header = None)

/Users/ericandresen/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [38]:
seekers.head()

,0,1,2,3,4
0,Extracted from the UNHCR Population Statistics...,United Nations High Commissioner for Refugees,NaN,NaN,NaN
1,Date extracted: 2015-11-26 04:22:38 +01:00,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,Country / territory of asylum/residence,Origin,Year,Month,Value
4,Australia,Afghanistan,1999,January,8


In [39]:
#make headers correct
seekers.columns = list(seekers.iloc[3,:])

In [42]:
seekers.head()

,Destination,Origin,Year,Month,Value
0,Australia,Afghanistan,1999,January,8
1,Australia,Afghanistan,1999,February,10
2,Australia,Afghanistan,1999,March,25
3,Australia,Afghanistan,1999,April,25
4,Australia,Afghanistan,1999,May,7


In [41]:
#drop rows that aren't data, make columns more understandable, and reset index
seekers = seekers.iloc[4:]
seekers.rename(columns = {'Country / territory of asylum/residence': 'Destination'}, inplace = True)
seekers.reset_index(drop = True, inplace = True)

In [43]:
#make date column that Tableau can parse
month_num = {
    'January' : 1,
    'February': 2,
    'March' : 3,
    'April' : 4,
    'May' : 5,
    'June' : 6,
    'July' : 7,
    'August' : 8,
    'September' : 9,
    'October' : 10,
    'November' : 11,
    'December' : 12,
}

seekers['Date'] = seekers.Month.map(month_num).astype(str) + '/1/' + seekers.Year.astype(str)

In [44]:
seekers.head(3)

,Destination,Origin,Year,Month,Value,Date
0,Australia,Afghanistan,1999,January,8,1/1/1999
1,Australia,Afghanistan,1999,February,10,2/1/1999
2,Australia,Afghanistan,1999,March,25,3/1/1999


In [22]:
seekers['Country / territory of asylum/residence'].value_counts()

Canada                                                  25693
USA (INS/DHS)                                           19734
USA (EOIR)                                              18559
Germany                                                 18147
France                                                  17870
United Kingdom of Great Britain and Northern Ireland    17709
Sweden                                                  17035
Belgium                                                 16336
Switzerland                                             15404
Netherlands                                             14548
Norway                                                  11978
Austria                                                 11905
Australia                                               11220
Spain                                                   10477
Denmark                                                  9420
Ireland                                                  9131
Greece  

It look like USA (EOIR) is a subset of USA (INS/DHS). The department of Justice sometimes handles asylum decisions where a person was rejected in the first review but is under a credible threat of persecution. You can read more about the US immigration process [here](http://trac.syr.edu/immigration/reports/159/)

I will label the DHS values USA for geocoding by Tableau. It would be interesting to drill into these results with a different data set to see if the US is rejecting more/less people in mortal danger.

In [45]:
seekers.Destination = seekers.Destination.replace('USA (INS/DHS)', 'USA')
seekers = seekers[seekers.Destination != 'USA (EOIR)']

In [47]:
seekers.Destination.value_counts()

Canada                                                  25693
USA                                                     19734
Germany                                                 18147
France                                                  17870
United Kingdom of Great Britain and Northern Ireland    17709
Sweden                                                  17035
Belgium                                                 16336
Switzerland                                             15404
Netherlands                                             14548
Norway                                                  11978
Austria                                                 11905
Australia                                               11220
Spain                                                   10477
Denmark                                                  9420
Ireland                                                  9131
Greece                                                   7616
Finland 

In [49]:
seekers.to_csv('unhcr_asylumseekers_monthly_clean.csv')